<a href="https://colab.research.google.com/github/anwishn/ESAA/blob/main/project3_%EC%98%81%ED%99%94%EA%B4%80%EA%B0%9D%EC%88%98%EC%98%88%EC%B8%A1_%ED%9A%8C%EA%B7%80.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[YB 미니 프로젝트(3) 공지]

- 관련 사이트 : https://dacon.io/competitions/open/235536/codeshare
- 과제 : 미니 프로젝트(1)에 이어서, 한 학기동안 배운 회귀 모델들 중 하나 이상을 해당 데이터에 적합.

- 조건 : 하이퍼 파라미터 튜닝 포함

* 분석 파일을 1) 제출 후 리더보드에서 점수 확인 2) github에 업로드하여 점수와 깃허브 링크를 과제함에 댓글로 달아주시면 됩니다. 점수 고득점자 1위 조원분들께는 소정의 상품 지급과 우수학회원 점수 가산(1위 조 전원 +3점)이 있을 예정입니다.

* 12/2, 12/9 금요일 yb 발표 세션 이후, 미니 프로젝트 회의하신 다음, 당일 자정까지 미니 프로젝트 보고서를 제출해주시면 됩니다. (부담없이 간단하게 작성해주세요)

* 분석 파일 제출 후에 설문조사를 통해 조별로 우수조원을 선정할 예정입니다.

* 제출기한 : 2022.12.30 (금) 23:59

# **1. 라이브러리 및 데이터**

In [ ]:
# matplotlib 한글 깨짐방지
# 아래코드 실행후 '런타임 다시시작' 후 모듈 불러오기
# 참고 블로그: https://teddylee777.github.io/colab/colab-korean 
# !sudo apt-get install -y fonts-nanum
# !sudo fc-cache -fv
# !rm ~/.cache/matplotlib -rf

In [ ]:
# 기본 모듈 불러오기
import pandas as pd 
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.rc('font', family='NanumBarunGothic') #matplotlib 한글 깨짐 방지

import re
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# 데이터 불러오기
train = pd.read_csv('/content/drive/MyDrive/data/movie/movies_train.csv')
test = pd.read_csv('/content/drive/MyDrive/data/movie/movies_test.csv')
submission= pd.read_csv('/content/drive/MyDrive/data/movie/submission.csv')

# **2. 탐색적 자료분석(EDA)**

## 1) 일부 데이터 미리보기

>1. movies_train.csv / movies_test.csv

* title : 영화의 제목

* distributor : 배급사

* genre : 장르

* release_time : 개봉일

* time : 상영시간(분)

* screening_rat : 상영등급

* director : 감독이름

* dir_prev_bfnum : 해당 감독이 이 영화를 만들기 전 제작에 참여한 영화에서의 평균 관객수(단 관객수가 알려지지 않은 영화 제외)

* dir_prev_num : 해당 감독이 이 영화를 만들기 전 제작에 참여한 영화의 개수(단 관객수가 알려지지 않은 영화 제외)

* num_staff : 스텝수

* num_actor : 주연배우수

* box_off_num : 관객수


>2.  submission.csv (제출 파일 형식)

2010년대 한국에서 개봉한 한국영화 600개에 대한 감독, 이름, 상영등급, 관객수 등의 정보가 담긴 데이터

In [ ]:
train.head()

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num
0,개들의 전쟁,롯데엔터테인먼트,액션,2012-11-22,96,청소년 관람불가,조병옥,NaN,0,91,2,23398
1,내부자들,(주)쇼박스,느와르,2015-11-19,130,청소년 관람불가,우민호,1161602.5,2,387,3,7072501
2,은밀하게 위대하게,(주)쇼박스,액션,2013-06-05,123,15세 관람가,장철수,220775.2,4,343,4,6959083
3,나는 공무원이다,(주)NEW,코미디,2012-07-12,101,전체 관람가,구자홍,23894.0,2,20,6,217866
4,불량남녀,쇼박스(주)미디어플렉스,코미디,2010-11-04,108,15세 관람가,신근호,1.0,1,251,2,483387


In [ ]:
test.head()

,title,distributor,genre,release_time,time,screening_rat,director,dir_prev_bfnum,dir_prev_num,num_staff,num_actor
0,용서는 없다,시네마서비스,느와르,2010-01-07,125,청소년 관람불가,김형준,300529.0,2,304,3
1,아빠가 여자를 좋아해,(주)쇼박스,멜로/로맨스,2010-01-14,113,12세 관람가,이광재,342700.2,4,275,3
2,하모니,CJ 엔터테인먼트,드라마,2010-01-28,115,12세 관람가,강대규,4206610.7,3,419,7
3,의형제,(주)쇼박스,액션,2010-02-04,116,15세 관람가,장훈,691342.0,2,408,2
4,평행 이론,CJ 엔터테인먼트,공포,2010-02-18,110,15세 관람가,권호영,31738.0,1,380,1


## 2) 데이터 요약

In [ ]:
print(train.shape)
print(test.shape)
print(submission.shape)

(600, 12)
(243, 11)
(243, 2)


In [ ]:
train.info()   # 피처의 타입과 Null 값 알아보기

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           600 non-null    object 
 1   distributor     600 non-null    object 
 2   genre           600 non-null    object 
 3   release_time    600 non-null    object 
 4   time            600 non-null    int64  
 5   screening_rat   600 non-null    object 
 6   director        600 non-null    object 
 7   dir_prev_bfnum  270 non-null    float64
 8   dir_prev_num    600 non-null    int64  
 9   num_staff       600 non-null    int64  
 10  num_actor       600 non-null    int64  
 11  box_off_num     600 non-null    int64  
dtypes: float64(1), int64(5), object(6)
memory usage: 56.4+ KB


In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 243 entries, 0 to 242
Data columns (total 11 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   title           243 non-null    object 
 1   distributor     243 non-null    object 
 2   genre           243 non-null    object 
 3   release_time    243 non-null    object 
 4   time            243 non-null    int64  
 5   screening_rat   243 non-null    object 
 6   director        243 non-null    object 
 7   dir_prev_bfnum  107 non-null    float64
 8   dir_prev_num    243 non-null    int64  
 9   num_staff       243 non-null    int64  
 10  num_actor       243 non-null    int64  
dtypes: float64(1), int64(4), object(6)
memory usage: 21.0+ KB


## 3) 데이터의 요약통계량 확인

In [ ]:
pd.options.display.float_format = '{:.1f}'.format

In [ ]:
train.describe()   # 숫자형 column들의 기술 통게량

,time,dir_prev_bfnum,dir_prev_num,num_staff,num_actor,box_off_num
count,600.0,270.0,600.0,600.0,600.0,600.0
mean,100.9,1050442.9,0.9,151.1,3.7,708181.8
std,18.1,1791408.3,1.2,165.7,2.4,1828005.9
min,45.0,1.0,0.0,0.0,0.0,1.0
25%,89.0,20380.0,0.0,17.0,2.0,1297.2
50%,100.0,478423.6,0.0,82.5,3.0,12591.0
75%,114.0,1286568.6,2.0,264.0,4.0,479886.8
max,180.0,17615314.0,5.0,869.0,25.0,14262766.0


In [ ]:
test.describe()

,time,dir_prev_bfnum,dir_prev_num,num_staff,num_actor
count,243.0,107.0,243.0,243.0,243.0
mean,109.8,891669.5,0.9,159.3,3.5
std,124.0,1217341.5,1.2,163.0,2.1
min,40.0,34.0,0.0,0.0,0.0
25%,91.0,62502.0,0.0,18.0,2.0
50%,104.0,493120.0,0.0,105.0,3.0
75%,114.5,1080849.6,1.0,282.0,4.0
max,2015.0,6173099.5,6.0,776.0,16.0


## 4) 결측치 확인 및 데이터 전처리

* **결측치** 처리 - **dir_prev_bfnum**(해당 감독이 이 영화를 만들기 전 제작에 참여한 영화에서의 평균 관객수)

In [ ]:
train.isna().sum()     # NaN 값 있는지 확인

title               0
distributor         0
genre               0
release_time        0
time                0
screening_rat       0
director            0
dir_prev_bfnum    330
dir_prev_num        0
num_staff           0
num_actor           0
box_off_num         0
dtype: int64

In [ ]:
test.isna().sum()

title               0
distributor         0
genre               0
release_time        0
time                0
screening_rat       0
director            0
dir_prev_bfnum    136
dir_prev_num        0
num_staff           0
num_actor           0
dtype: int64

=> train과 test 모두 dir_prev_bfnum 변수에 결측값이 존재한다.

In [ ]:
# 결측치가 많은 데이터 제거
train = train.drop(['dir_prev_bfnum'],axis = 1)
test =  test.drop(['dir_prev_bfnum'],axis = 1)

* **director**(감독명), **title**(제목), **dir_prev_num**(해당 감독이 이 영화를 만들기 전 제작에 참여한 영화의 개수) 제거

In [ ]:
# director, title, num_actor 의미없어 제거/ dir_prev_num은 0이 많아 제거
train = train.drop(['director','title','dir_prev_num'],axis = 1)
test = test.drop(['director','title','dir_prev_num'],axis = 1)

* **상영 시간**(time) 이상치 처리

* **배급사**(distributor) 전처리

In [ ]:
train['distributor'] = train.distributor.str.replace("(주)", '')
test['distributor'] = test.distributor.str.replace("(주)", '')

In [ ]:
train['distributor'] = [re.sub(r'[^0-9a-zA-Z가-힣]', '', x) for x in train.distributor]
test['distributor'] = [re.sub(r'[^0-9a-zA-Z가-힣]', '', x) for x in test.distributor]

In [ ]:
def get_dis(x) :
    if 'CJ' in x or 'CGV' in x :
        return 'CJ'
    elif '쇼박스' in x :
        return '쇼박스'
    elif 'SK' in x :
        return 'SK'
    elif '리틀빅픽' in x :
        return '리틀빅픽처스'
    elif '스폰지' in x :
        return '스폰지'
    elif '싸이더스' in x :
        return '싸이더스'
    elif '에이원' in x :
        return '에이원'
    elif '마인스' in x :
        return '마인스'
    elif '마운틴픽' in x :
        return '마운틴픽처스'
    elif '디씨드' in x :
        return '디씨드'
    elif '드림팩트' in x :
        return '드림팩트'
    elif '메가박스' in x :
        return '메가박스'
    elif '마운틴' in x :
        return '마운틴'
    else :
        return x

In [ ]:
train['distributor'] = train.distributor.apply(get_dis)
test['distributor'] = test.distributor.apply(get_dis)

* **배급사**별 영화 관객수 중위값 기준으로 배급사 랭크 인코딩

In [ ]:
tr_nm_rank = train.groupby('distributor').box_off_num.median().reset_index(name = 'num_rank').sort_values(by = 'num_rank')
tr_nm_rank

,distributor,num_rank
110,인피니티엔터테인먼트,2.0
15,고구마공작소,8.0
52,사람과사람들,42.0
97,위드시네마,46.0
19,나우콘텐츠,54.0
...,...,...
113,전망좋은영화사,1214237.0
105,이십세기폭스코리아,1422844.0
56,쇼박스,2138560.0
84,영구아트무비,2541603.0


In [ ]:
tr_nm_rank['num_rank'] = [i + 1 for i in range(tr_nm_rank.shape[0])]
tr_nm_rank

,distributor,num_rank
110,인피니티엔터테인먼트,1
15,고구마공작소,2
52,사람과사람들,3
97,위드시네마,4
19,나우콘텐츠,5
...,...,...
113,전망좋은영화사,143
105,이십세기폭스코리아,144
56,쇼박스,145
84,영구아트무비,146


In [ ]:
train = pd.merge(train, tr_nm_rank, how = 'left')
test = pd.merge(test, tr_nm_rank, how = 'left')
test.fillna(0, inplace = True)

* **장르**별 영화 관객수 평균값으로 랭크 인코딩

In [ ]:
train.groupby('genre').box_off_num.mean().sort_values()

genre
뮤지컬         6627.0
다큐멘터리      67172.3
서스펜스       82611.0
애니메이션     181926.7
멜로/로맨스    425968.0
미스터리      527548.2
공포        590832.5
드라마       625689.8
코미디      1193914.0
SF       1788345.7
액션       2203974.1
느와르      2263695.1
Name: box_off_num, dtype: float64

In [ ]:
train['genre_rank'] = train.genre.map({'뮤지컬' : 1, '다큐멘터리' : 2, '서스펜스' : 3, '애니메이션' : 4, '멜로/로맨스' : 5,
                                      '미스터리' : 6, '공포' : 7, '드라마' : 8, '코미디' : 9, 'SF' : 10, '액션' : 11, '느와르' : 12})
test['genre_rank'] = test.genre.map({'뮤지컬' : 1, '다큐멘터리' : 2, '서스펜스' : 3, '애니메이션' : 4, '멜로/로맨스' : 5,
                                      '미스터리' : 6, '공포' : 7, '드라마' : 8, '코미디' : 9, 'SF' : 10, '액션' : 11, '느와르' : 12})

* **개봉일**(release_time)

In [ ]:
train['season'] = 0
test['season'] = 0

In [ ]:
train['month'] = train['release_time'].apply(lambda x: int(x[5:7]))
train.head()

,distributor,genre,release_time,time,screening_rat,num_staff,num_actor,box_off_num,num_rank,genre_rank,season,month
0,롯데엔터테인먼트,액션,2012-11-22,96,청소년 관람불가,91,2,23398,134,11,0,11
1,쇼박스,느와르,2015-11-19,130,청소년 관람불가,387,3,7072501,145,12,0,11
2,쇼박스,액션,2013-06-05,123,15세 관람가,343,4,6959083,145,11,0,6
3,NEW,코미디,2012-07-12,101,전체 관람가,20,6,217866,140,9,0,7
4,쇼박스,코미디,2010-11-04,108,15세 관람가,251,2,483387,145,9,0,11


In [ ]:
test['month'] = test['release_time'].apply(lambda x: int(x[5:7]))
test.head()

,distributor,genre,release_time,time,screening_rat,num_staff,num_actor,num_rank,genre_rank,season,month
0,시네마서비스,느와르,2010-01-07,125,청소년 관람불가,304,3,132.0,12,0,1
1,쇼박스,멜로/로맨스,2010-01-14,113,12세 관람가,275,3,145.0,5,0,1
2,CJ,드라마,2010-01-28,115,12세 관람가,419,7,141.0,8,0,1
3,쇼박스,액션,2010-02-04,116,15세 관람가,408,2,145.0,11,0,2
4,CJ,공포,2010-02-18,110,15세 관람가,380,1,141.0,7,0,2


In [ ]:
train.month.value_counts()

11    72
10    60
9     60
5     57
8     55
3     52
7     49
12    49
4     47
1     39
2     31
6     29
Name: month, dtype: int64

In [ ]:
def func(month):
  if month in [3, 4, 5]:
    return '봄'
  elif month in [6, 7, 8]:
    return '여름'
  elif month in [9, 10, 11]:
    return '가을'
  elif month in [12, 1, 2]:
    return '겨울'

In [ ]:
train['season'] = train['month'].apply(lambda x: func(x))
train.head()

,distributor,genre,release_time,time,screening_rat,num_staff,num_actor,box_off_num,num_rank,genre_rank,season,month
0,롯데엔터테인먼트,액션,2012-11-22,96,청소년 관람불가,91,2,23398,134,11,가을,11
1,쇼박스,느와르,2015-11-19,130,청소년 관람불가,387,3,7072501,145,12,가을,11
2,쇼박스,액션,2013-06-05,123,15세 관람가,343,4,6959083,145,11,여름,6
3,NEW,코미디,2012-07-12,101,전체 관람가,20,6,217866,140,9,여름,7
4,쇼박스,코미디,2010-11-04,108,15세 관람가,251,2,483387,145,9,가을,11


In [ ]:
test['season'] = test['month'].apply(lambda x: func(x))
test.head()

,distributor,genre,release_time,time,screening_rat,num_staff,num_actor,num_rank,genre_rank,season,month
0,시네마서비스,느와르,2010-01-07,125,청소년 관람불가,304,3,132.0,12,겨울,1
1,쇼박스,멜로/로맨스,2010-01-14,113,12세 관람가,275,3,145.0,5,겨울,1
2,CJ,드라마,2010-01-28,115,12세 관람가,419,7,141.0,8,겨울,1
3,쇼박스,액션,2010-02-04,116,15세 관람가,408,2,145.0,11,겨울,2
4,CJ,공포,2010-02-18,110,15세 관람가,380,1,141.0,7,겨울,2


In [ ]:
# release_time을 season과 month으로 바꿨으니 삭제
train = train.drop(['release_time'], axis=1)
test = test.drop(['release_time'], axis=1)

* **상영 등급**(screening_rat)

In [ ]:
train.screening_rat.value_counts()

청소년 관람불가    204
15세 관람가     202
12세 관람가     102
전체 관람가       92
Name: screening_rat, dtype: int64

In [ ]:
train['screening_rat'] = train['screening_rat'].apply(lambda x : 0 \
    if x == '전체 관람가' else 1 if x == '12세 관람가' else 2 if x == '15세 관람가' \
        else 3)
test['screening_rat'] = test['screening_rat'].apply(lambda x : 0 \
    if x == '전체 관람가' else 1 if x == '12세 관람가' else 2 if x == '15세 관람가' \
        else 3)

* **스텝 수**(num_staff)를 4분위수로 변환

In [ ]:
train['num_staff'].describe()

count   600.0
mean    151.1
std     165.7
min       0.0
25%      17.0
50%      82.5
75%     264.0
max     869.0
Name: num_staff, dtype: float64

In [ ]:
def func(quant):
  if quant <= 17:
    return 1
  elif 17< quant <= 82.5:
    return 2
  elif 82.5 < quant <= 264:
    return 3
  else:
    return 4

In [ ]:
train['staff_quantile'] = train['num_staff'].apply(lambda x: func(x))
train.head()   

,distributor,genre,time,screening_rat,num_staff,num_actor,box_off_num,num_rank,genre_rank,season,month,staff_quantile
0,롯데엔터테인먼트,액션,96,3,91,2,23398,134,11,가을,11,3
1,쇼박스,느와르,130,3,387,3,7072501,145,12,가을,11,4
2,쇼박스,액션,123,2,343,4,6959083,145,11,여름,6,4
3,NEW,코미디,101,0,20,6,217866,140,9,여름,7,2
4,쇼박스,코미디,108,2,251,2,483387,145,9,가을,11,3


In [ ]:
test['staff_quantile'] = test['num_staff'].apply(lambda x: func(x))
test.head() 

,distributor,genre,time,screening_rat,num_staff,num_actor,num_rank,genre_rank,season,month,staff_quantile
0,시네마서비스,느와르,125,3,304,3,132.0,12,겨울,1,4
1,쇼박스,멜로/로맨스,113,1,275,3,145.0,5,겨울,1,4
2,CJ,드라마,115,1,419,7,141.0,8,겨울,1,4
3,쇼박스,액션,116,2,408,2,145.0,11,겨울,2,4
4,CJ,공포,110,2,380,1,141.0,7,겨울,2,4


In [ ]:
# num_staff을 staff_quantile로 바꿨으니 삭제
train = train.drop(['num_staff'], axis=1)
test = test.drop(['num_staff'], axis=1)

## 5) 모델링 데이터 구성

* 타겟 값인 관객수를 로그변환

In [ ]:
X = train[['num_rank', 'genre_rank','time', 'staff_quantile','num_actor', 'screening_rat','season']]
y = np.log1p(train.box_off_num)

In [ ]:
X = pd.get_dummies(columns = ['season'], data = X)

In [ ]:
target = test[['num_rank', 'genre_rank','time', 'staff_quantile','num_actor', 'screening_rat','season']]

In [ ]:
target = pd.get_dummies(columns = ['season'], data = target)

# 데이터 분할

# 회귀 모델 적용

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
#from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
#from ngboost import NGBRegressor

In [ ]:
kf = KFold(n_splits =  10, shuffle = True, random_state = 42)

In [ ]:
gbm = GradientBoostingRegressor(random_state = 42)

rmse_list = []
gb_pred = np.zeros((test.shape[0]))
for tr_idx, val_idx in kf.split(X, y) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    gbm.fit(tr_x, tr_y)
    
    pred = np.expm1([0 if x < 0 else x for x in gbm.predict(val_x)])
    sub_pred = np.expm1([0 if x < 0 else x for x in gbm.predict(target)])
    rmse = np.sqrt(mean_squared_error(val_y, pred))
    
    rmse_list.append(rmse)
    
    gb_pred += (sub_pred / 10)

np.mean(rmse_list)

1075097.7426386264

In [ ]:
lgbm = LGBMRegressor(random_state = 518)

rmse_list = []
lgb_pred = np.zeros((test.shape[0]))
for tr_idx, val_idx in kf.split(X, y) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    lgbm.fit(tr_x, tr_y)
    
    pred = np.expm1([0 if x < 0 else x for x in lgbm.predict(val_x)])
    sub_pred = np.expm1([0 if x < 0 else x for x in lgbm.predict(target)])
    rmse = np.sqrt(mean_squared_error(val_y, pred))
    
    rmse_list.append(rmse)
    
    lgb_pred += (sub_pred / 10)

np.mean(rmse_list)

1382656.2678328545

In [ ]:
xgb = XGBRegressor(random_state = 518)

rmse_list = []
xgb_pred = np.zeros((test.shape[0]))
for tr_idx, val_idx in kf.split(X, y) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    xgb.fit(tr_x, tr_y)
    
    pred = np.expm1([0 if x < 0 else x for x in xgb.predict(val_x)])
    sub_pred = np.expm1([0 if x < 0 else x for x in xgb.predict(target)])
    rmse = np.sqrt(mean_squared_error(val_y, pred))
    
    rmse_list.append(rmse)
    
    xgb_pred += (sub_pred / 10)

np.mean(rmse_list)

[02:57:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:57:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:57:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:57:43] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:57:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:57:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:57:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
[02:57:44] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of re

1079559.8497626008

In [ ]:
rf = RandomForestRegressor(random_state = 518)

rmse_list = []
rf_pred = np.zeros((test.shape[0]))
for tr_idx, val_idx in kf.split(X, y) :
    tr_x, tr_y = X.iloc[tr_idx], y.iloc[tr_idx]
    val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
    
    rf.fit(tr_x, tr_y)
    
    pred = np.expm1([0 if x < 0 else x for x in rf.predict(val_x)])
    sub_pred = np.expm1([0 if x < 0 else x for x in rf.predict(target)])
    rmse = np.sqrt(mean_squared_error(val_y, pred))
    
    rmse_list.append(rmse)
    
    rf_pred += (sub_pred / 10)

np.mean(rmse_list)

994990.3025414664

In [ ]:
pred = (xgb_pred + lgb_pred + rf_pred + gb_pred) / 4

# 제출 데이터 뽑아내기

In [ ]:
submission['box_off_num'] = pred
submission

,title,box_off_num
0,용서는 없다,1312746.3
1,아빠가 여자를 좋아해,1648734.6
2,하모니,1092189.4
3,의형제,2295990.6
4,평행 이론,2044296.3
...,...,...
238,해에게서 소년에게,15454.4
239,울보 권투부,5795.0
240,어떤살인,61.1
241,말하지 못한 비밀,1369.5


In [ ]:
submission.to_csv('project3_베이스라인.csv',index = False)